In [58]:
import hashlib
import base64
import json
import time
import ssl
import requests
import urllib
url_preffix='https://api.ai.qq.com/fcgi-bin/'

def setParams(array, key, value):
    array[key] = value

def genSignString(parser):
    uri_str = ''
    
    for key in sorted(parser.keys()):
        if key == 'app_key':
            continue
        uri_str += "%s=%s&" % (key, urllib.parse.quote(str(parser[key]), safe = ''))########
    sign_str = uri_str + 'app_key=' + parser['app_key']

    hash_md5 = hashlib.md5(sign_str.encode("utf-8"))
    return hash_md5.hexdigest().upper()


class AiPlat(object):
    def __init__(self, app_id= '2160782408', app_key='dqJl6DLMAYKSZ9Md'):
        self.app_id = app_id
        self.app_key = app_key
        self.data = {}

    def invoke(self, params):
        req=requests.get(url=self.url,params=params)
#         print(req)
#         print(req.text)
#         print(req.json)
        
        try:
            rsp = req.text
#             print('111111',rsp)
#             print(rsp)
            str_rsp = rsp
            dict_rsp = json.loads(str_rsp)
            return dict_rsp
        except urllib.error.URLError as e:
            dict_error = {}
            if hasattr(e, "code"):
                dict_error = {}
                dict_error['ret'] = -1
                dict_error['httpcode'] = e.code
                dict_error['msg'] = "sdk http post err"
                return dict_error
            if hasattr(e,"reason"):
                dict_error['msg'] = 'sdk http post err'
                dict_error['httpcode'] = -1
                dict_error['ret'] = -1
                return dict_error
        else:
            dict_error = {}
            dict_error['ret'] = -1
            dict_error['httpcode'] = -1
            dict_error['msg'] = "system error"
            return dict_error


    def getOcrGeneralocr(self, image):
        self.url = url_preffix + 'ocr/ocr_generalocr'
        setParams(self.data, 'app_id', self.app_id)
        setParams(self.data, 'app_key', self.app_key)
        setParams(self.data, 'time_stamp', int(time.time()))
        setParams(self.data, 'nonce_str', int(time.time()))
        image_data = base64.b64encode(image)
        setParams(self.data, 'image', image_data)
        sign_str = genSignString(self.data)
        setParams(self.data, 'sign', sign_str)
        return self.invoke(self.data)

    def getNlpTextTrans(self, text, type):
#         print(text,type)
#         time.sleep(0.5)

        self.data = {}
        self.url = url_preffix + 'nlp/nlp_texttrans'
        setParams(self.data, 'app_id', self.app_id)
        setParams(self.data, 'app_key', self.app_key)
        setParams(self.data, 'time_stamp', int(time.time()))
        setParams(self.data, 'nonce_str', int(time.time()))
        setParams(self.data, 'text', text)
        setParams(self.data, 'type', type)
        sign_str = genSignString(self.data)
        setParams(self.data, 'sign', sign_str)
        return self.invoke(self.data)

    def getAaiWxAsrs(self, chunk, speech_id, end_flag, format_id, rate, bits, seq, chunk_len, cont_res):
        self.url = url_preffix + 'aai/aai_wxasrs'
        setParams(self.data, 'app_id', self.app_id)
        setParams(self.data, 'app_key', self.app_key)
        setParams(self.data, 'time_stamp', int(time.time()))
        setParams(self.data, 'nonce_str', int(time.time()))
        speech_chunk = base64.b64encode(chunk)
        setParams(self.data, 'speech_chunk', speech_chunk)
        setParams(self.data, 'speech_id', speech_id)
        setParams(self.data, 'end', end_flag)
        setParams(self.data, 'format', format_id)
        setParams(self.data, 'rate', rate)
        setParams(self.data, 'bits', bits)
        setParams(self.data, 'seq', seq)
        setParams(self.data, 'len', chunk_len)
        setParams(self.data, 'cont_res', cont_res)
        sign_str = genSignString(self.data)
        setParams(self.data, 'sign', sign_str)
        return self.invoke(self.data)


In [59]:
aaaaa=AiPlat()

In [60]:
def txt_chuli(string):
    ss=string
    for i in [',',"'",'#','!',"！",'.','。','？','?','[',']','{','}','【','】','`','~','@','%','&','，']:
        ss=ss.replace(i,'')
    t=ss.split(' ')
    for i in range(len(t)):
        if i!=0:
            t[i]=t[i].capitalize()
    s=''
    for i in t:
        s+=i
    return s

In [62]:
with open(r'data\relations_3.csv','r',encoding='utf-8') as f:
    with open(r'data\rrr3.csv','a+',encoding='utf-8') as ff:
        count=0
        for line in f:
            count+=1
            print('count:',count)
            row = line.split(',')
            flag = 1
            while flag:
                type = 0
                rsp = aaaaa.getNlpTextTrans(row[0],type)
                print(rsp)
                if rsp['ret'] == 0:
                    first=rsp['data']['org_text']
                    second=txt_chuli(rsp['data']['trans_text'])
                    ff.write(first+','+second+'\n')
                    flag=0
            



count: 1
{'ret': 0, 'msg': 'ok', 'data': {'type': 0, 'org_text': '属性', 'trans_text': 'Properties'}}
count: 2
{'ret': 0, 'msg': 'ok', 'data': {'type': 0, 'org_text': '描述', 'trans_text': 'Description'}}
count: 3
{'ret': -16, 'msg': 'system busy, please try again later', 'data': {'type': 0, 'org_text': '', 'trans_text': ''}}
{'ret': 0, 'msg': 'ok', 'data': {'type': 0, 'org_text': '标签', 'trans_text': 'Label'}}
count: 4
{'ret': 0, 'msg': 'ok', 'data': {'type': 0, 'org_text': '中文名', 'trans_text': 'Chinese name'}}
count: 5
{'ret': -16, 'msg': 'system busy, please try again later', 'data': {'type': 0, 'org_text': '', 'trans_text': ''}}
{'ret': 0, 'msg': 'ok', 'data': {'type': 0, 'org_text': '是否含防腐剂', 'trans_text': 'Does it contain preservatives?'}}
count: 6
{'ret': 0, 'msg': 'ok', 'data': {'type': 0, 'org_text': '主要食用功效', 'trans_text': 'Main edible efficacy'}}
count: 7
{'ret': -16, 'msg': 'system busy, please try again later', 'data': {'type': 0, 'org_text': '', 'trans_text': ''}}
{'ret': 0, '

From cffi callback <function _verify_callback at 0x0000028A405D9400>:
Traceback (most recent call last):
  File "D:\Programspecial\Anaconda\lib\site-packages\OpenSSL\SSL.py", line 306, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='api.ai.qq.com', port=443): Max retries exceeded with url: /fcgi-bin/nlp/nlp_texttrans?app_id=2160782408&app_key=dqJl6DLMAYKSZ9Md&time_stamp=1607871815&nonce_str=1607871815&text=%E6%8B%BC+%E9%9F%B3&type=0&sign=675368B0071824E141926610E90354ED (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [72]:
import requests
import json
import time
def fanyi(word="你没有输入要翻译的内容"):
    url ="http://fanyi.youdao.com/translate?smartresult=dict&smartresult=rule&smartresult=ugc&sessionFrom=null"
    data={
        'type':"AUTO",
        'i':word,
        "doctype":"json",
        "keyfrom":"fanyi.web",
        "ue":"UTF-8",
        "action":"FY_BY_CLICKBUTTON",
        "typoResult":"true"
    }

    response = requests.post(url,data=data)
    return (json.loads(response.text)["translateResult"][0][0]["tgt"])
#i=0
with open(r'data\relations_3.csv',"r",encoding='utf-8') as f:
    with open(r"data\rela.csv","a+",encoding='utf-8') as ff:
        for row in f:
            time.sleep(0.15)
            line=row.rstrip('\n').split(',')
            first=line[0]
            second=''
            tmp=fanyi(first).split(' ')
            for v in range(len(tmp)):
                if v!=0:
                    tmp[v]=tmp[v].capitalize()

                second+=tmp[v]
            print(second[0],end='')
            ff.write(first+','+second+'\n')


JSONDecodeError: Expecting value: line 1 column 1 (char 0)